In [29]:
import os
import numpy as np
import subprocess as sp

In [30]:
mod_dir = r'C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys'

In [31]:
hgs_mod_dir = os.path.join(mod_dir,'hgs')
model_name = 'lys'
grok_file_path = os.path.join(hgs_mod_dir,model_name + '_e.grok')

In [32]:
grok_file_stem = model_name + '_e'

In [33]:
def CreateCoupledModelDir(mod_dir):
    # Create directories if they don't exist
    coupled_mod_dir = os.path.join(mod_dir,'coupled')
    try:
        os.mkdir(coupled_mod_dir)
    except:
        print(coupled_mod_dir + ' already exists')
    # Create hgs subdirectory
    coupled_mod_hgs_dir = os.path.join(coupled_mod_dir,'hgs')
    try:
        os.mkdir(coupled_mod_hgs_dir)
    except:
        print(coupled_mod_hgs_dir + ' already exists')
    # Create dssat subdirectory
    coupled_mod_dssat_dir = os.path.join(coupled_mod_dir,'dssat')
    try:
        os.mkdir(coupled_mod_dssat_dir)
    except:
        print(coupled_mod_dssat_dir + ' already exists')
    return coupled_mod_dir,coupled_mod_hgs_dir,coupled_mod_dssat_dir

In [ ]:
def GetSpinUpHeadsOutputFile(hgs_mod_dir,grok_file_stem):
    # Identify spin up grok name
    

In [34]:
def GetStandaloneGrokLines(grok_file_path):
    # Read grok file
    with open(grok_file_path,'r') as file_in:
        lines = file_in.readlines()
    return lines

In [35]:

def GetStandaloneGrokPrecSeries(standalone_grok_lines):
    # Get start index
    start = standalone_grok_lines.index('!!--Begin Precipitation Time Series Section--\n')
    # Get end index
    end = standalone_grok_lines.index('!!--End Precipitation Time Series Section--\n')
    # Get P Block
    plines = standalone_grok_lines[start:end]
    # Get p series
    start = plines.index('    time value table\n')
    end = plines.index('    end\n')
    # Get time series lines
    tslines = plines[start+1:end]
    # Get Daily P Series
    p = [float(x.split(' ')[5].strip()) for x in tslines]
    # Get end day
    end_day = len(p)
    return p, end_day


In [36]:
def CreateDailyCoupledGrokFileDay0(lines,day,p):
    ## P Section
    # Get start index
    pstart = lines.index('!!--Begin Precipitation Time Series Section--\n')
    # Get end index
    pend = lines.index('!!--End Precipitation Time Series Section--\n')
    # Build P entry
    pentry = f'    time value table\n    0.0 {P[day]:.2f}\n    end\n'
    ## Output Section
    # Get start index
    ostart = lines.index('!!--Begin Output Times Section--\n')
    # Get end index
    oend = lines.index('!!--End Output Times Section--\n')
    new_lines = lines[:pstart+1]+[pentry]+lines[pend:ostart+1]+['1.0\n']+lines[oend:]
    return new_lines

In [37]:
def CreateDailyCoupledGrokFileDayN(lines,day,p,grok_file_stem):
    ## IC Section
    # Get start index
    icstart = lines.index('!!--Begin Initial Head Section--\n')
    # Get end index
    icend = lines.index('!!--End Initial Head Section--\n')
    # Build IC
    icentry = '! Set initial heads from day n-1\nchoose nodes all\n\ninitial head from output file\n{0}day{1}o.head_pm.0001\n\nclear chosen nodes\n'.format(grok_file_stem,day-1)
    ## P Section
    # Get start index
    pstart = lines.index('!!--Begin Precipitation Time Series Section--\n')
    # Get end index
    pend = lines.index('!!--End Precipitation Time Series Section--\n')
    # Build P entry
    pentry = f'    time value table\n    0.0 {p[day]:.2f}\n    end\n'
    ## Output Section
    # Get start index
    ostart = lines.index('!!--Begin Output Times Section--\n')
    # Get end index
    oend = lines.index('!!--End Output Times Section--\n')
    new_lines = lines[:icstart+1]+[icentry]+lines[icend:pstart+1]+[pentry]+lines[pend:ostart+1]+['1.0\n']+lines[oend:]
    return new_lines

In [38]:
def WriteCoupledGrokFile(new_lines,day,coupled_mod_hgs_dir,grok_file_stem):
    new_grok_name = grok_file_stem + 'day{}'.format(day) + '.grok'
    new_grok_path = os.path.join(coupled_mod_hgs_dir,new_grok_name)
    with open(new_grok_path,'w') as file:
        for entry in new_lines:
            file.write(entry)

In [39]:
## Build Coupled Model
# Create Directory Structure
cmod_dir,cmod_hgs_dir,cmod_dssat_dir = CreateCoupledModelDir(mod_dir)
# Get standalone model grok lines and Prec series
standalone_grok_lines = GetStandaloneGrokLines(grok_file_path)
P, End_Day = GetStandaloneGrokPrecSeries(standalone_grok_lines)
# Iterate through days to build daily hgs models
for day in np.arange(0,End_Day):
    # Day 0 model
    if day == 0:
        # Build text lines
        new_cgrok_lines = CreateDailyCoupledGrokFileDay0(standalone_grok_lines,day,P)
        # Write out
        WriteCoupledGrokFile(new_cgrok_lines,day,cmod_hgs_dir,grok_file_stem)
    # All other Day models
    else:
        # Build text lines
        new_cgrok_lines = CreateDailyCoupledGrokFileDayN(standalone_grok_lines,day,P,grok_file_stem)
        # Write out
        WriteCoupledGrokFile(new_cgrok_lines,day,cmod_hgs_dir,grok_file_stem)

C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled already exists
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs already exists
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\dssat already exists


## Controller

In [40]:
def CreateControllerBatchFile(coupled_mod_dir,coupled_mod_hgs_dir):
    controller_path = os.path.join(coupled_mod_hgs_dir,'Controller.bat')
    with open(controller_path,'w') as file:
        file.write('cd {}\ngrok > out_g.txt\nphgs > out_h.txt\n'.format(coupled_mod_hgs_dir))
    out_path = os.path.join(coupled_mod_hgs_dir,'out.txt')
    with open(out_path,'w') as file:
        file.write('')


In [41]:
def RunCoupledModelHGSDaily(day,coupled_mod_hgs_dir,grok_file_stem):
    grok_name = grok_file_stem + 'day{}'.format(day)
    # First, update batch.pfx
    print('Updating batch.pfx')
    batch_pfx_path = os.path.join(coupled_mod_hgs_dir,'batch.pfx')
    with open(batch_pfx_path,'w') as file:
        file.write(grok_name)
    # Then run Controller
    print('Running Model')
    sp.run(['Controller.bat'])

In [42]:
## Run Daily models
# First, create controller batch file
CreateControllerBatchFile(cmod_dir,cmod_hgs_dir)
# Change to coupled model directory
os.chdir(cmod_hgs_dir)
# Iterate through days to run daily hgs models
for day in np.arange(0,End_Day):
    print(day)
    print(os.getcwd())
    RunCoupledModelHGSDaily(day,cmod_hgs_dir,grok_file_stem)

0
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
1
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
2
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
3
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
4
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
5
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
6
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
7
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
8
C:\Users\southa0000\Documents\HGS-DSSAT\HGS-DSSAT\examples\lys\coupled\hgs
Updating batch.pfx
Running Model
9
C:\Users